In [1]:
include("../basis.jl")
include("../identity.jl")
include("../basic_gates.jl")
using Compat.Test
using BenchmarkTools

In [2]:
### pretty interface
for GATE in [:xgate, :ygate, :zgate, :ndiaggate, :diaggate, :controlled_U1, :cnotgate, :projector, :toffoligate, :swapgate]
    @eval $GATE(num_bit::Int, params...) = $GATE(basis(num_bit), params...)
end

In [3]:
function xgate(basis::Vector{DInt}, bits::Ints)
    norder = flip(basis, bits)
    PermuteMultiply(norder+1, ones(length(basis)))
end

function ygate(basis::Vector{DInt}, bits::Ints)
    norder = flip(basis, bits)
    #vals = [-im*(-1)^reduce(+,takebit(b, bits)) for b in basis]
    #vals = mapreduce(bit->map(x->x==0?-im:im, takebit(basis, bit)), .*, bits)
    vals = mapreduce(bit->im.*(2.*takebit(basis, bit).-1), .*, bits)
    PermuteMultiply(norder+1, vals)
end

function zgate(basis::Vector{DInt}, bits::Ints)
    #vals = [(-1)^reduce(+,takebit(b, bits)) for b in basis]
    vals = mapreduce(bit->1.-2.*takebit(basis, bit), .*, bits)
    PermuteMultiply(basis+1, vals)
end

zgate (generic function with 2 methods)

In [4]:
############################ TODO ################################
# arbitrary off-diagonal single qubit gate
# e.g. X, Y
function ndiaggate(basis::Vector{DInt}, gate::PermuteMultiply, bits::Ints)
    norder = flip(basis, bits)
    vals = mapreduce(bit->exp.(im*phi*(2.*takebit(basis, bit).-1)), .*, bits)
    PermuteMultiply(norder+1, vals)
end

# arbitrary diagonal single qubit gate
# e.g. Z, Rz(θ)
function diaggate(basis::Vector{DInt}, gate::Diagonal, bits::Ints)
    vals = mapreduce(bit->exp.(im*phi*(2.*takebit(basis, bit).-1)), .*, bits)
    PermuteMultiply(basis+1, vals) # or Diagonal(vals) ?
end

#TODO?
# arbituary dense single qubit gate: SparseMatrixCSC
# e.g. Rx(θ), Ry(θ), Rot(θ1,θ2,θ3)
#function densegate(bits::Ints, basis::Vector{DInt})
#end

# arbituary control PermuteMultiply gate: SparseMatrixCSC
# swap gate
function swapgate(basis::Vector{DInt}, b1::Int, b2::Int)
end

swapgate (generic function with 2 methods)

In [5]:
⊗ = kron
function superkron(num_bit::Int, gates::Vector{T}, locs::Vector{Int}) where T<:AbstractMatrix
    locs = num_bit - locs + 1
    order = sortperm(locs)
    _wrap_identity(gates[order], diff(vcat([0], locs[order], [num_bit+1])) - 1)
end

# kron, and wrap matrices with identities.
function _wrap_identity(data_list::Vector{T}, num_bit_list::Vector{Int}) where T<:AbstractMatrix
    length(num_bit_list) == length(data_list) + 1 || throw(ArgumentError())
    reduce((x,y)->x ⊗ y[1] ⊗ II(1<<y[2]), II(1 << num_bit_list[1]), zip(data_list, num_bit_list[2:end]))
end

_wrap_identity (generic function with 1 method)

In [6]:
@assert superkron(4, [PAULI_X, PAULI_Y], [3,2]) == II(2) ⊗ PAULI_X ⊗ PAULI_Y ⊗ II(2)

In [7]:
# a function projecting space into target space.
function projector(basis::Vector{DInt}, cbits::Int)
    # Assigned: LW
end

projector (generic function with 2 methods)

In [8]:
#@test projector(basis(6), [2, -3, -5]) == superkron(6, [P0, P1, P1], [2, 3, 5])

In [9]:
general_controlled_gates(num_bit::Int, projectors::Vector{Tp}, cbits::Vector{Int}, gates::Vector{Tg}, locs::Vector{Int}) where {Tg<:AbstractMatrix, Tp<:AbstractMatrix} = II(1<<num_bit) - superkron(num_bit, projectors, cbits) + superkron(num_bit, vcat(projectors, gates), vcat(cbits, locs))

general_controlled_gates (generic function with 1 method)

In [10]:
CNOT = PermuteMultiply(Complex128[1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0])
TOFFOLI = kron(P0, II(4)) + kron(P1, CNOT)
@test general_controlled_gates(2, [P1], [2], [PAULI_X], [1]) == CNOT

Test Passed


In [11]:
@test zgate(4, [1,2,3]) == superkron(4, [PAULI_Z, PAULI_Z, PAULI_Z], [1,2,3])

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##716#717)(::Array{Int64,1}, ::Array{Int64,1}) at ./deprecated.jl:346
 [3] _mapreduce(::##40#42{Array{Int64,1}}, ::Base.##716#717, ::IndexLinear, ::Array{Int64,1}) at ./reduce.jl:273
 [4] zgate(::Array{Int64,1}, ::Array{Int64,1}) at ./In[3]:16
 [5] zgate(::Int64, ::Array{Int64,1}) at ./In[2]:3
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/leo/.julia/v0.6/IJulia/src/execute_request.jl:158
 [8] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/leo/.julia/v0.6/Compat/src/Compat.jl:385
 [9] eventloop(::ZMQ.Socket) at /home/leo/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[11], in expression starting on line 1
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##716#717)(::Array{Int64,1}, ::Array{Int64,1}) at ./d

Test Passed


In [12]:
# Assigned: LW
# control gate for single qubit gate
function controlled_U1(basis::Vector{DInt}, gate::PermuteMultiply, b1::Int, b2::Int)
end
function controlled_U1(basis::Vector{DInt}, gate::SparseMatrixCSC, b1::Int, b2::Int)
end
function controlled_U1(basis::Vector{DInt}, gate::Diagonal, b1::Int, b2::Int)
end
function controlled_U1(basis::Vector{DInt}, gate::StridedMatrix, b1::Int, b2::Int)
end

controlled_U1 (generic function with 5 methods)

In [13]:
# CNOT/CZ may be further accelerated.
czgate(basis::Vector{DInt}, b1::Int, b2::Int) = controlled_U1(basis, PAULI_Z, b1, b2)

czgate (generic function with 1 method)

In [14]:
############# CNOT Gate as an example ##################
function cnotgate(basis::Vector{DInt}, b1::Int, b2::Int)
    @views order = xor.(basis, (basis .& bmask(b1)) .<< (b2-b1))
    PermuteMultiply(order+1, ones(Int, length(basis)))
end
println(cnotgate(basis(2), 2, 1))

PermuteMultiply
- (1) * 1
- (2) * 1
- (4) * 1
- (3) * 1



In [20]:
@test cnotgate(basis(2), 2, 1) == [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]
@test general_controlled_gates(16, [P1], [7], [PAULI_X], [3]) == cnotgate(basis(16), 7, 3)

Test Passed


In [23]:
bss = basis(16);
############# uncomment to run benchmarks ##################
#@benchmark cnotgate($(bss), 7, 3)
#@benchmark general_controlled_gates(16, [P1], [7], [PAULI_X], [3])

In [ ]:
toffoligate(basis::Vector{DInt}, b1::Int, b2::Int, b3::Int) = multicontrolled_U1(basis, PAULI_X, [b1, b2], b3)

In [28]:
# Assigned: LW
# multi-control gate for single qubit gate
function multicontrolled_U1(basis::Vector{DInt}, gate::PermuteMultiply, cbits::Vector{Int}, b2::Int)
end
# repeat for other matrix formats ...

multicontrolled_U1 (generic function with 1 method)

In [29]:
# TODO
# future, arbituary two site gate support
# probabily focus is enough? or, use projection decomposition?

In [12]:
#@test toffligate(3, 2, 3, 1) == TOFFOLI

8×8 SparseMatrixCSC{Complex{Float64},Int64} with 8 stored entries:
  [1, 1]  =  1.0+0.0im
  [2, 2]  =  1.0+0.0im
  [3, 3]  =  1.0+0.0im
  [4, 4]  =  1.0+0.0im
  [5, 5]  =  1.0+0.0im
  [6, 6]  =  1.0+0.0im
  [8, 7]  =  1.0+0.0im
  [7, 8]  =  1.0+0.0im